<a href="https://colab.research.google.com/github/lucasnserra/Market-Basket-Analysis-in-Python/blob/main/3%C2%B0_Market_Basket_The_Apriori_Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importo Librerias
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import permutations
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori,association_rules

In [2]:
#Cargo dataset y convierto en lista 
gifts = pd.read_csv('https://raw.githubusercontent.com/lucasnserra/Market-Basket-Analysis-in-Python/main/online_retail.csv')
print(gifts.head(2))
#cantidad de facturas
Q = len(gifts['InvoiceNo'].unique())
#cantidad de Items
I = len(gifts['Description'].unique())

print('Cantidad de facturas unicas ', format(Q))
print('Cantidad de Items unicos ', format(I))
gifts['Description'] = gifts.Description.astype('str')

  InvoiceNo StockCode                      Description
0    562583    35637A  IVORY STRING CURTAIN WITH POLE 
1    562583    35638A    PINK AND BLACK STRING CURTAIN
Cantidad de facturas unicas  9709
Cantidad de Items unicos  3461


In [3]:
#Preprocessing (OneHot) / debo crear una Lista de Listas
#Agrupo por factura los distintos productos comprados
gifts = gifts.groupby(['InvoiceNo'], as_index = False).agg({'Description': '|'.join})
#Modifico a str por errores con Float
List = gifts['Description'].apply(str)
#Creo la lista de productos separando los campos.
List = list(gifts['Description'].apply(lambda t: t.split('|')))


In [4]:
#Creo el encoder y lo entreno
encoder = TransactionEncoder().fit(List)
#Hago el OneHot (Crea una matriz de True o False si aparece el atributo en la fila)
onehot = encoder.transform(List)
#Creo DF 
onehot = pd.DataFrame(onehot,columns=encoder.columns_)
onehot.head()

,4 PURPLE FLOCK DINNER CANDLES,50'S CHRISTMAS GIFT BAG LARGE,DOLLY GIRL BEAKER,I LOVE LONDON MINI BACKPACK,NINE DRAWER OFFICE TIDY,OVAL WALL MIRROR DIAMANTE,RED SPOT GIFT BAG LARGE,SET 2 TEA TOWELS I LOVE LONDON,SPACEBOY BABY GIFT SET,TRELLIS COAT RACK,...,wet boxes,wet pallet,wet rusty,wet?,wrongly coded 20713,wrongly coded 23343,wrongly coded-23343,wrongly marked,wrongly marked 23343,wrongly marked carton 22804
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Algoritmo Apriori (Pruning), que nos ayudará a reducir la complejidad de grandes conjuntos de datos al eliminar conjuntos de elementos de bajo soporte. (No elimina reglas de asociacion). El algoritmo Apriori está estructurado en torno a la idea de que debemos conservar elementos que son frecuentes, es decir, exceder un nivel mínimo de soporte. El principio a priori establece que los subconjuntos de conjuntos frecuentes también deben ser frecuentes.

In [5]:
frequent_itemsets_1 = apriori(onehot,min_support = 0.0015,max_len =2, use_colnames = True)
print(len(frequent_itemsets_1))
print(frequent_itemsets_1.head())

240842
    support                          itemsets
0  0.002369  ( 4 PURPLE FLOCK DINNER CANDLES)
1  0.012257  ( 50'S CHRISTMAS GIFT BAG LARGE)
2  0.012772              ( DOLLY GIRL BEAKER)
3  0.007416    ( I LOVE LONDON MINI BACKPACK)
4  0.002884        ( NINE DRAWER OFFICE TIDY)


In [6]:
frequent_itemsets_2 = apriori(onehot,min_support = 0.001,max_len =2, use_colnames = True)
print(len(frequent_itemsets_2))
print(frequent_itemsets_2.head())

420324
    support                          itemsets
0  0.002369  ( 4 PURPLE FLOCK DINNER CANDLES)
1  0.012257  ( 50'S CHRISTMAS GIFT BAG LARGE)
2  0.012772              ( DOLLY GIRL BEAKER)
3  0.007416    ( I LOVE LONDON MINI BACKPACK)
4  0.002884        ( NINE DRAWER OFFICE TIDY)


In [7]:
# Compute all association rules for frequent_itemsets_1
rules_1 = association_rules(frequent_itemsets_1)
# Compute all association rules for frequent_itemsets_2
rules_2 = association_rules(frequent_itemsets_2, metric = 'confidence', min_threshold = 0.0015)
# Print the number of association rules generated
#print(len(rules_1), len(rules_2))
rules_1.head().sort_values(by = 'support')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
2,(DAISY HAIR BAND),(3 RAFFIA RIBBONS 50'S CHRISTMAS ),0.001648,0.019466,0.001545,0.937500,48.159722,0.001513,15.688536
0,(PAIR OF PINK FLOWER CLUSTER SLIDE),(12 PENCIL SMALL TUBE WOODLAND),0.001854,0.020805,0.001648,0.888889,42.723872,0.001609,8.812751
3,(PAIR OF PINK FLOWER CLUSTER SLIDE),(3 RAFFIA RIBBONS 50'S CHRISTMAS ),0.001854,0.019466,0.001648,0.888889,45.662551,0.001612,8.824802
4,(SET 10 CARDS DINKY TREE 17076),(3 RAFFIA RIBBONS 50'S CHRISTMAS ),0.001648,0.019466,0.001648,1.000000,51.370370,0.001616,inf
1,(3 RAFFIA RIBBONS VINTAGE CHRISTMAS),(3 RAFFIA RIBBONS 50'S CHRISTMAS ),0.009785,0.019466,0.008446,0.863158,44.340741,0.008255,7.165437


In [8]:
rules_2.head().sort_values(by = 'support')

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(4 IVORY DINNER CANDLES SILVER FLOCK),( 4 PURPLE FLOCK DINNER CANDLES),0.007004,0.002369,0.001236,0.176471,74.493606,0.001219,1.211409
1,( 4 PURPLE FLOCK DINNER CANDLES),(4 IVORY DINNER CANDLES SILVER FLOCK),0.002369,0.007004,0.001236,0.521739,74.493606,0.001219,2.076265
2,(4 PINK DINNER CANDLE SILVER FLOCK),( 4 PURPLE FLOCK DINNER CANDLES),0.001957,0.002369,0.001236,0.631579,266.608696,0.001231,2.707856
3,( 4 PURPLE FLOCK DINNER CANDLES),(4 PINK DINNER CANDLE SILVER FLOCK),0.002369,0.001957,0.001236,0.521739,266.608696,0.001231,2.086817
4,( 50'S CHRISTMAS GIFT BAG LARGE),( RED SPOT GIFT BAG LARGE),0.012257,0.009682,0.002781,0.226891,23.434919,0.002662,1.280955


In [11]:
# aplico filtro avanzado
filtered_rules = rules_1[(rules_1['antecedent support'] > 0.002) &
						(rules_1['consequent support'] > 0.01) &
						(rules_1['confidence'] > 0.6) &
						(rules_1['lift'] > 2.50)]

filtered_rules[['antecedents','consequents']]

,antecedents,consequents
1,(3 RAFFIA RIBBONS VINTAGE CHRISTMAS),(3 RAFFIA RIBBONS 50'S CHRISTMAS )
7,(CERAMIC STRAWBERRY TRINKET TRAY),(6 RIBBONS EMPIRE )
13,(ANGEL DECORATION PAINTED ZINC ),(STAR DECORATION PAINTED ZINC )
14,(ANGEL DECORATION PAINTED ZINC ),(WOODEN HEART CHRISTMAS SCANDINAVIAN)
15,(ANGEL DECORATION PAINTED ZINC ),(WOODEN STAR CHRISTMAS SCANDINAVIAN)
...,...,...
315,(WOVEN FROST CUSHION COVER),(SUKI SHOULDER BAG)
316,(WOVEN ROSE GARDEN CUSHION COVER ),(SUKI SHOULDER BAG)
317,(WOVEN SUNSET CUSHION COVER ),(SUKI SHOULDER BAG)
318,(VINTAGE BEAD COSMETIC BAG ),(TOAST ITS - FAIRY FLOWER)
